In [1]:
import pandas as pd
import os
from glob import glob

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

arquivos_csv = []
arquivos_csv.extend(glob(os.path.join(raiz, 'ouro_npk_*.csv')))
arquivos_csv.sort()

df_balanco = pd.DataFrame()

fontes = ('Carcaça Bovina',
          'Dejetos Galináceos',
          'Dejetos Suínos',
          'Deposição Atmosférica',
          'Fertilizante orgânico vinhaça',
          'Fertilizante Sintético',
          'Fixação Bio N',
          'Outras Produções Agrícolas',
          'Principais Produções Agrícolas',
          'Sementes'
          )

In [3]:
df_balanco = pd.DataFrame()

for arquivo, fonte in zip(arquivos_csv, fontes):
    df = (pd.read_csv(arquivo)
          .assign(valor=lambda v: v.valor/1000,
                  fluxo=lambda x: 'Saída' if arquivo in (arquivos_csv[0], arquivos_csv[7], arquivos_csv[8]) else 'Entrada')
        )
    df_balanco = (pd.concat([df_balanco, df], ignore_index=True)
                  .query('uf != 1')
                  .sort_values(
                      by=['ano', 'uf', 'fonte'],
                      key=lambda x: x.map({
                          'Fertilizantes Sintéticos':0,
                          'Fertilizante orgânico vinhaça':1,
                          'Dejetos galináceos':2,
                          'Dejetos suínos':3,
                          'Fixação biológica de N':4,
                          'Sementes':5,
                          'Deposição Atmosférica':6,
                          'Principais produções agrícolas':7,
                          'Outras produções agrícolas':8,
                          'Carcaça Bovina':9
                      })
                      )
                  .reset_index(drop=True)
                  )

In [4]:
df_balanco = (df_balanco.groupby(['uf', 'ano'])
    [['fonte', 'elemento', 'valor', 'fluxo']]
    .apply(lambda x: x)
    .sort_values(
        by=['elemento', 'uf'],
        key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2})
        )
        .reset_index()
        .drop('level_2', axis='columns')
        .convert_dtypes()
                )

df_balanco.to_csv(f'{raiz}/ouro_npk-balanco.csv', index=False)